#Waymo Open Dataset Tutorial

- Website: https://waymo.com/open
- GitHub: https://github.com/waymo-research/waymo-open-dataset

This tutorial demonstrates how to use the Waymo Open Dataset with two frames of data. Visit the [Waymo Open Dataset Website](https://waymo.com/open) to download the full dataset.

To use, open this notebook in [Colab](https://colab.research.google.com).

Uncheck the box "Reset all runtimes before running" if you run this colab directly from the remote kernel. Alternatively, you can make a copy before trying to run it by following "File > Save copy in Drive ...".



## Install waymo_open_dataset package

In [1]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r2.0
!pip3 install --upgrade pip

Cloning into 'waymo-od'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 753 (delta 0), reused 0 (delta 0), pack-reused 749
Receiving objects: 100% (753/753), 14.16 MiB | 42.04 MiB/s, done.
Resolving deltas: 100% (480/480), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
  remotes/origin/r1.0-tf2.0
  remotes/origin/r1.2
error: pathspec 'remotes/origin/r2.0' did not match any file(s) known to git.
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)


In [10]:
!rm /content/frame*
!wget https://github.com/aabramovrepo/uploads/raw/master/waymo_dataset/frame.bin

--2020-03-26 20:33:31--  https://github.com/aabramovrepo/uploads/raw/master/waymo_dataset/frame.bin
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aabramovrepo/uploads/master/waymo_dataset/frame.bin [following]
--2020-03-26 20:33:31--  https://raw.githubusercontent.com/aabramovrepo/uploads/master/waymo_dataset/frame.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5114145 (4.9M) [application/octet-stream]
Saving to: ‘frame.bin’

frame.bin           100%[===================>]   4.88M  31.3MB/s    in 0.2s    

2020-03-26 20:33:31 (31.3 MB/s) - ‘frame.bin’ saved [5114145/5114145]



In [11]:
!ls -lh /content

total 4.9M
-rw-r--r-- 1 root root 4.9M Mar 26 20:33 frame.bin
drwxr-xr-x 1 root root 4.0K Mar 18 16:23 sample_data
drwxr-xr-x 9 root root 4.0K Mar 26 20:26 waymo-od


In [12]:
!pip3 install waymo-open-dataset-tf-2-1-0==1.2.0
import os
import tensorflow as tf
import math
import numpy as np
import itertools

tf.compat.v1.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

## Read one frame

Each file in the dataset is a sequence of frames ordered by frame start timestamps. We have extracted two frames from the dataset to demonstrate the dataset format.

In [0]:
with tf.io.gfile.GFile('/content/frame.bin', 'rb') as f:
  github_frame = f.read()
  github_frame_decoded = open_dataset.Frame()
  github_frame_decoded.ParseFromString(bytearray(github_frame))
  frame = github_frame_decoded


In [0]:
(range_images, camera_projections,
 range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(
    frame)

##Point Cloud Conversion

In [0]:
points, cp_points = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose)
points_ri2, cp_points_ri2 = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose,
    ri_index=1)

# 3d points in vehicle frame.
points_all = np.concatenate(points, axis=0)
points_all_ri2 = np.concatenate(points_ri2, axis=0)
# camera projection corresponding to each point.
cp_points_all = np.concatenate(cp_points, axis=0)
cp_points_all_ri2 = np.concatenate(cp_points_ri2, axis=0)

points_r1_2 = np.concatenate([points_all, points_all_ri2], axis=0)

###Compare

L2 norm

In [31]:
np.linalg.norm(points_all)

8515.677

Output from github

In [24]:
!wget https://github.com/aabramovrepo/uploads/raw/master/waymo_dataset/points_all.npy
!rm -rf github || true
!mkdir github && cp /content/points_all.npy /content/github/points_all.npy && cd github  
!ls /content/github

--2020-03-26 20:37:30--  https://github.com/aabramovrepo/uploads/raw/master/waymo_dataset/points_all.npy
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aabramovrepo/uploads/master/waymo_dataset/points_all.npy [following]
--2020-03-26 20:37:31--  https://raw.githubusercontent.com/aabramovrepo/uploads/master/waymo_dataset/points_all.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2013752 (1.9M) [application/octet-stream]
Saving to: ‘points_all.npy.1’

points_all.npy.1    100%[===================>]   1.92M  --.-KB/s    in 0.1s    

2020-03-26 20:37:31 (14.8 MB/s) - ‘points_all.npy.1’ saved [201

In [32]:
pc = np.load('/content/github/points_all.npy')
np.linalg.norm(pc)

8515.677